# CW_Preprocessing:

Date:23/01/23

This file contains all the preprocessing and various SQLite functions used throughout this project.

Information regarding how data is handled:

    - Module names come from the file names, please make sure that the first 8 characters of the file
      reflect the module name (e.g. 22COA122)
      
    - We removed all students that had an attendance of 0 across all modules. This is because although some
      students were recorded as absent, with an attendance of 0 we're unable to distinguish if they were 
      enrolled on the modules or if they had changed modules/not enrolled on the course anymore.

## Imports 

In [1]:

import pandas as pd
import numpy as np  
import sqlite3
from pathlib import Path
import os



## Functions 

### Dataframe preparations

In [2]:

def csv_data(): 
    """
    Takes CSV data files from the moduleregister_data folder found next to this script file and loads it into
    a dictionairy. This also extracts the module name via the file name.
    """
    file_loc = Path('moduleregister_data')
    csv_dict = {}
    for filename in os.listdir(file_loc):
        filename_path = f'{file_loc}/{filename}'
        csv_dict[filename[:8]] = pd.read_csv(filename_path).to_dict()
    return csv_dict

    
def session_data(csv_dict):
    """
    Takes the CSV dictionary of module data and extracts the session data (e.g time, location, type) placing them
    into seperate pd dataframes.
    """
    csv_dict_sessions = {}
    for i in csv_dict.keys():
        module_df = pd.DataFrame.from_dict(csv_dict[i]).columns.to_list()
        module_df = pd.DataFrame(module_df, columns=['details'])
        module_df = module_df['details'].str.split('\n', expand=True)
        module_df.columns = ['semester_week', 'date', 'time', 'type', 'room', '']
        module_df = module_df.iloc[1:, :-1]
        module_df[['day_of_week', 'date1']] = module_df.date.str.split(' ', expand=True)
        module_df['date'] = module_df['date1']
        module_df = module_df.iloc[:, :-1]
        module_df['date'] = module_df['date'].str.strip('()')
        module_df['date'] = pd.to_datetime(module_df['date'], format='%d-%m-%Y')
        module_df[['start_time', 'end_time']] = module_df.time.str.split(' - ', expand=True)
        module_df['start_time'] = pd.to_datetime(module_df['start_time']).dt.time
        module_df['end_time'] = pd.to_datetime(module_df['end_time']).dt.time
        module_df = module_df.drop(columns=['time'])
        module_df = module_df[['semester_week', 'date', 'start_time', 'end_time', 'day_of_week', 'type', 'room']]
        module_df = module_df.replace('', np.nan, regex=True)
        module_df[['semester', 'week']] = module_df.semester_week.str.split('.', expand=True)
        module_df['semester'] = module_df['semester'].str.strip('S')
        module_df['week'] = module_df['week'].str.strip('W')
        module_df[['semester', 'week']] = module_df[['semester', 'week']].apply(pd.to_numeric)
        module_df['semester_week'] = module_df['semester_week'].astype(str) + '.'
        module_df.semester_week = module_df.semester_week + \
                            module_df.groupby('semester_week').cumcount().add(1).astype(str) + '_' + str(i)
        module_df = module_df.rename(columns={'semester_week': 'session_id'})
        module_df['session_id'] = module_df['session_id'].str.replace('.', '_', regex=True)
        module_df['module'] = i
        csv_dict_sessions[i + str('session')] = module_df
    return csv_dict_sessions


def student_data(csv_dict):
    """
    Takes the CSV dictionary of module data and extracts the student attendance data placing them into 
    seperate pd dataframes.
    """
    csv_dict_student = {}
    for i in csv_dict.keys():
        student_df = pd.DataFrame.from_dict(csv_dict[i])
        student_df = student_df.rename(columns = lambda x : str(x)[:5]) # shorten column names
        temp_df = student_df.columns.to_series().groupby(student_df.columns) # as there was duplicate names, made each unique by adding .x
        student_df = student_df.add_suffix('_')
        student_df.columns = np.where(temp_df.transform('size')>0, 
                      student_df.columns + temp_df.cumcount().add(1).astype(str), 
                      student_df.columns)
        student_df.replace('Ex', None, inplace = True) # Converting attendance to True,False,Na
        student_df.replace({'GPS':True,'X':False}, inplace = True)
        student_df.columns = student_df.columns.str.replace('.','_',regex = True)
        student_df = student_df.add_suffix('_' + str(i))
        student_df = student_df.rename(columns={student_df.columns[0]: 'sid'})
        student_df = student_df.dropna(subset = student_df.columns.difference(['sid']), how = 'all') # remove rows and columns that are all NaN
        student_df = student_df.dropna(axis = 1, how = 'all')
        student_df = student_df[student_df.iloc[:, 1:].sum(axis=1) >= 1] # remove any students that has 0 attendance
        csv_dict_student[i] = student_df
    return csv_dict_student


### SQL Database preparations

In [3]:

def write_to_db(dfData, TableName):
    """
    Takes a dataframe and table name and creates a new sql table (or replaces) which is saved to 
    the predetermined database file.
    """
    conn = sqlite3.connect('CWDatabase.db')
    dfData.to_sql('sql_' + TableName, conn, if_exists='replace',index = False) # Table name can't start with numbers
    conn.close()

    
def load_data(c_df_dict):
    """
    Loops data dictionary through write_to_db using the key as the table names. 
    """
    for i in c_df_dict.keys():
        sql_df = pd.DataFrame.from_dict(c_df_dict[i])
        write_to_db(sql_df, i) 
    return()


def read_db_data(tablename):
    """
    Reads SQLite database table, requires table name (e.g module code)
    """
    conn = sqlite3.connect('CWDatabase.db')
    new_table_name = "sql_" + tablename 
    df = pd.read_sql("""SELECT * FROM %s;""" % new_table_name, conn)
    return(df)


### Creating module sesssion SQL tables

In [4]:

def complete_sessions():
    """
    Takes all session tables, joins them, then writes them to the SQL database as complete_sessions.
    """
    conn = sqlite3.connect('CWDatabase.db')
    session_keys = session_data(csv_data()).keys() 
    merge_value = pd.DataFrame(columns=['session_id',
                                        'date',
                                        'start_time',
                                        'end_time',
                                        'day_of_week',
                                        'type',
                                        'room',
                                        'semester',
                                        'week',
                                        'module'])
    for i in session_keys:
        my_value = read_db_data(i)
        merge_value = merge_value.merge(my_value, on = ['session_id',
                                                        'date',
                                                        'start_time',
                                                        'end_time',
                                                        'day_of_week',
                                                        'type',
                                                        'room',
                                                        'semester',
                                                        'week',
                                                        'module'], how = 'outer')
    sql_df = pd.DataFrame(merge_value)
    write_to_db(sql_df, 'complete_sessions')


def att_module(module):
    """
    Takes module code, reads SQLite database and calculates the modules attendence per session
    for that module. 
    """
    att = pd.DataFrame(read_db_data(module))
    att_m = pd.DataFrame(att.iloc[:, 1:].sum(axis = 0))
    att_count = att.iloc[:, :1].count(axis = 0)
    att_m.columns = ["num_att"] # number of students attended
    att_m["no_student"] = att_count[0] - 1 # total number registered for module
    att_m["att_pct"] = ((att_m["num_att"] / att_m["no_student"])*100).round()
    att_m = att_m.reset_index().rename(columns ={"index":"session_id"})
    return(att_m)


def sql_att_module_join(module):
    """
    Takes module code, reads SQLite database and calculates the modules attendence per session
    for that module and join's it with the sessions details (e.g Start time, type).
    """
    att_pct_df = att_module(module)
    conn = sqlite3.connect('CWDatabase.db')
    session_table = "sql_" + module + "session"
    tablename = module + "_att"
    write_to_db(att_pct_df, tablename)
    sqlq = """SELECT {}.*, {}.num_att, {}.no_student, {}.att_pct
              FROM {} INNER JOIN {}
              ON {}.session_id = {}.session_id""".format(session_table,
                                                  "sql_" + tablename,
                                                  "sql_" + tablename,
                                                  "sql_" + tablename,
                                                  session_table,
                                                  "sql_" + tablename,
                                                  session_table,
                                                  "sql_" + tablename)
    s_m_att = pd.read_sql(sqlq, conn)
    return s_m_att


def get_complete_session():
    conn = sqlite3.connect('CWDatabase.db')
    sqlq = """SELECT *
              FROM sql_complete_sessions
              """
    db = pd.read_sql(sqlq, conn)
    df = pd.DataFrame(db)
    return(df)


### Creating and calculating student attendaence SQL tables

In [5]:

def complete_sid_attendence():
    """
    Takes all student data tables, joins them, then writes them to the SQL database as complete_attendence.
    """
    conn = sqlite3.connect('CWDatabase.db')
    tables = student_data(csv_data()).keys()
    merge_value = pd.DataFrame(columns = ["sid"])
    for i in tables:
        df = read_db_data(i)
        merge_value = merge_value.merge(df, on = "sid", how = "outer")
    sql_df = pd.DataFrame(merge_value)
    write_to_db(sql_df, "complete_attendence")
    
    
def sid_attendence(*sid):
    """
    Takes a sid (or multiple sid's) and returns their complete attendence.
    """
    conn = sqlite3.connect('CWDatabase.db')
    if len(sid) > 1: #checks how many sid's inputed as multiple requires the "IN" SQLite condition
        sqlq = """SELECT *
                  FROM sql_complete_attendence
                  WHERE sid in {}""".format(sid)
    else:
        sqlq = """SELECT *
              FROM sql_complete_attendence
              WHERE sid == {}""".format(sid[0])
    db = pd.read_sql(sqlq, conn)
    return(db)
    
    
def sid_attendence_pct(*sid):
    """
    Takes a sid (or multiple sid's) and returns their attendence count and percentage.
    """
    att = sid_attendence(*sid)
    att_s = pd.DataFrame(att.iloc[:,0])
    att_s['no_classes'] = att.notnull().sum(axis=1)
    att_s['no_att'] = pd.DataFrame(att.iloc[:, 1:].sum(axis = 1))
    att_s['att_pct'] = ((att_s['no_att'] / att_s['no_classes']) * 100).round()
    return(att_s)


def get_complete_attendence():
    conn = sqlite3.connect('CWDatabase.db')
    sqlq = """SELECT *
              FROM sql_complete_attendence
              """
    db = pd.read_sql(sqlq, conn)
    complete_att_df = pd.DataFrame(db)
    return(complete_att_df)


## Main Code

In [6]:
def main_preprocessing():
    """
    Essential code required to build database
    """
    # Reading and cleaning data
    raw_data = csv_data()
    sess_data = session_data(raw_data)
    stud_data = student_data(raw_data)

    # Writing to SQLite Database
    load_data(sess_data) 
    load_data(stud_data)
    complete_sessions()
    complete_sid_attendence()

    
# Calling main functions 
main_preprocessing()

## Testing 

In [7]:
def module_testing(module):
    """
    Testing module functions
    """
    return att_module(module), sql_att_module_join(module)
    
# Calling module testing functions using module 22COA111 to demonstrate outputs.
simple_22COA111, complete_22COA111 = module_testing("22COA111")

display(simple_22COA111)

display(complete_22COA111)


,session_id,num_att,no_student,att_pct
0,S1_W1_1_22COA111,143.0,191,75.0
1,S1_W1_2_22COA111,126.0,191,66.0
2,S1_W2_1_22COA111,138.0,191,72.0
3,S1_W2_3_22COA111,161.0,191,84.0
4,S1_W3_1_22COA111,120.0,191,63.0
5,S1_W3_3_22COA111,156.0,191,82.0
6,S1_W4_1_22COA111,121.0,191,63.0
7,S1_W4_3_22COA111,154.0,191,81.0
8,S1_W5_1_22COA111,126.0,191,66.0
9,S1_W5_3_22COA111,150.0,191,79.0


,session_id,date,start_time,end_time,day_of_week,type,room,semester,week,module,num_att,no_student,att_pct
0,S1_W1_1_22COA111,2022-10-03 00:00:00,09:00:00.000000,10:00:00.000000,Monday,Lecture,U020,1,1,22COA111,143.0,191,75.0
1,S1_W1_2_22COA111,2022-10-05 00:00:00,10:00:00.000000,11:00:00.000000,Wednesday,Lecture,SMB014,1,1,22COA111,126.0,191,66.0
2,S1_W2_1_22COA111,2022-10-12 00:00:00,10:00:00.000000,11:00:00.000000,Wednesday,Lecture,SMB014,1,2,22COA111,138.0,191,72.0
3,S1_W2_3_22COA111,2022-10-13 00:00:00,14:00:00.000000,15:00:00.000000,Thursday,Lecture,CC011,1,2,22COA111,161.0,191,84.0
4,S1_W3_1_22COA111,2022-10-19 00:00:00,10:00:00.000000,11:00:00.000000,Wednesday,Lecture,SMB014,1,3,22COA111,120.0,191,63.0
5,S1_W3_3_22COA111,2022-10-20 00:00:00,14:00:00.000000,15:00:00.000000,Thursday,Lecture,CC011,1,3,22COA111,156.0,191,82.0
6,S1_W4_1_22COA111,2022-10-26 00:00:00,10:00:00.000000,11:00:00.000000,Wednesday,Lecture,SMB014,1,4,22COA111,121.0,191,63.0
7,S1_W4_3_22COA111,2022-10-27 00:00:00,14:00:00.000000,15:00:00.000000,Thursday,Lecture,CC011,1,4,22COA111,154.0,191,81.0
8,S1_W5_1_22COA111,2022-11-02 00:00:00,10:00:00.000000,11:00:00.000000,Wednesday,Lecture,SMB014,1,5,22COA111,126.0,191,66.0
9,S1_W5_3_22COA111,2022-11-03 00:00:00,14:00:00.000000,15:00:00.000000,Thursday,Lecture,CC011,1,5,22COA111,150.0,191,79.0


In [8]:
def sid_att_testing(*sid):
    """
    Testing student attendance functions
    """
    return sid_attendence(*sid), sid_attendence_pct(*sid)
    
# Calling sid attendance testing functions using sid 10 and 211 to demonstrate outputs.
sid_10_211, sid_10_211_att_pct = sid_att_testing(10,211)

display(sid_10_211)

display(sid_10_211_att_pct)

,sid,S1_W1_1_22COA133,S1_W1_2_22COA133,S1_W1_3_22COA133,S1_W2_1_22COA133,S1_W2_2_22COA133,S1_W2_3_22COA133,S1_W3_1_22COA133,S1_W3_2_22COA133,S1_W3_3_22COA133,...,S1_W1_1_22COA111,S1_W1_2_22COA111,S1_W2_1_22COA111,S1_W2_3_22COA111,S1_W3_1_22COA111,S1_W3_3_22COA111,S1_W4_1_22COA111,S1_W4_3_22COA111,S1_W5_1_22COA111,S1_W5_3_22COA111
0,10,1.0,1.0,None,1.0,1.0,None,0.0,0.0,None,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1,211,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


,sid,no_classes,no_att,att_pct
0,10,45,26.0,58.0
1,211,33,30.0,91.0
